In [ ]:
import pandas as pd
from openai import AzureOpenAI

c:\Users\babeysinghe\AppData\Local\anaconda3\envs\deploy\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [ ]:
api_key = ... # your api key
api_version = '2024-06-01'
deployment_name = 'ChatGPT-3_5' # can be other models

In [ ]:
llm = AzureOpenAI( # this also can be OpenAI
    api_version=api_version,
    api_key=api_key
)

In [13]:
def build_prompt(abstract):
    return [{
                'role': 'system', 
                'content': '''You will be given an abstract of a scientific article, you are to extract a topic for the article given its abstract.
                    A topic will be few words, following is an example to how to extract topics.
                    
                    <abstract>:
                        Research and practice in climate risk reduction often view marginalized individuals through the lens of vulnerability. 
                        However, this perspective lacks specificity of which groups and needs should be incorporated, features narrow wealth-based conceptualization and provides 
                        insufficient operationalizable guidance for planning and implementation. This study highlights the theoretical and practical significance of a 
                        functional-based approach. It transcends the apparent differences among social groups, instead identifying their shared activity limitations 
                        and associated access and functional needs (AFNs) amid climate hazards. Those social groups generally include but not limited to people with 
                        disabilities, limited language proficiency, restricted mobility and economic disadvantage, pregnant women as well as children and seniors. 
                        We combine quantitative and qualitative analysis to investigate how and why local governments incorporate AFNs in their climate risk reduction. 
                        Based on hazard mitigation and climate adaptation plans across local governments in California, our results show that AFN inclusion is consistently 
                        predicted by AFN incorporation in higher-level plans, rather than the presence of AFN populations. Besides, plans embracing the functional-based approach 
                        achieve greater comprehensiveness and depth of AFN inclusion. We further highlight the commonalities and differences between the two types of plans and 
                        conclude with strategic and operational implications for risk reduction efforts.
                        Please only respond with the topic, nothing else
                    <topic>: climate, adaptation, change
                    '''
            }, 
            {
                'role': 'user', 
                'content': f'''<abstract>: {abstract}
<topic>: 
'''
            }]

In [14]:
def ask_chatgpt(abstract):
    response = llm.chat.completions.create(
        model='ChatGPT-3_5',
        messages=build_prompt(abstract=abstract),
    )
    return response.choices[0].message.content

In [9]:
data = (
    pd.read_feather('ESP.feather')
    .assign(authors=lambda df_x: df_x.authors.str.replace('<#>', ';'))[['authors', 'title', 'pub_date', 'abstract', 'cite_count', 'doi', 'pii', 'openaccess']]
)
data = data[~data.abstract.str.fullmatch('-1')].reset_index(drop=True)
data

,authors,title,pub_date,abstract,cite_count,doi,pii,openaccess
0,"Zhang, Fengxiu;Xiang, Tianyi",Attending to the unattended: Why and how do lo...,2024-12-31,Research and practice in climate risk reductio...,0,10.1016/j.envsci.2024.103892,S1462901124002260,false
1,"Yoshida, Yuki;Sitas, Nadia;Mannetti, Lelani;O’...",Beyond Academia: A case for reviews of gray li...,2024-12-31,Gray literature is increasingly considered to ...,0,10.1016/j.envsci.2024.103882,S1462901124002168,true
2,"Pietrzyk-Kaszyńska, Agata;Olszańska, Agnieszka",Of heroes and villains – How coalitions shape ...,2024-12-31,Policy narrative analyses provide important in...,0,10.1016/j.envsci.2024.103899,S1462901124002338,false
3,"Zurba, Melanie;Suchet-Pearson, Sandie;Bullock,...",Enhancing meaningful Indigenous leadership and...,2024-12-31,This is the first global empirical study that ...,0,10.1016/j.envsci.2024.103864,S1462901124001989,true
4,"Lemke, Leonard Kwhang-Gil;Beier, Julia;Hanger-...",Exploring procedural justice in stakeholder id...,2024-12-31,"In the face of complex societal challenges, st...",0,10.1016/j.envsci.2024.103900,S146290112400234X,true
...,...,...,...,...,...,...,...,...
3025,"Danila, Donald J",Estimating the abundance and egg production of...,2000-09-01,The winter flounder is a coastal flatfish with...,9,10.1016/S1462-9011(00)00059-9,S1462901100000599,false
3026,"Lorda, Ernesto;Danila, Donald J.;Miller, John D.",Application of a population dynamics model to ...,2000-09-01,A major concern of the Connecticut Department ...,7,10.1016/S1462-9011(00)00061-7,S1462901100000617,false
3027,"Jager, Henriëtte I;Lepla, Ken;Chandler, James;...",Population viability analysis of white sturgeo...,2000-09-01,Many utilities face conflicts between two goal...,13,10.1016/S1462-9011(00)00063-0,S1462901100000630,false
3028,"Farber, Stephen",Welfare-based ecosystem management: an investi...,2000-09-01,This paper investigates the use of welfare-bas...,1,10.1016/S1462-9011(00)00066-6,S1462901100000666,false


In [10]:
data.iloc[0].abstract

'Research and practice in climate risk reduction often view marginalized individuals through the lens of vulnerability. However, this perspective lacks specificity of which groups and needs should be incorporated, features narrow wealth-based conceptualization and provides insufficient operationalizable guidance for planning and implementation. This study highlights the theoretical and practical significance of a functional-based approach. It transcends the apparent differences among social groups, instead identifying their shared activity limitations and associated access and functional needs (AFNs) amid climate hazards. Those social groups generally include but not limited to people with disabilities, limited language proficiency, restricted mobility and economic disadvantage, pregnant women as well as children and seniors. We combine quantitative and qualitative analysis to investigate how and why local governments incorporate AFNs in their climate risk reduction. Based on hazard mi

In [15]:
ask_chatgpt(data.iloc[0].abstract)

'climate risk reduction, vulnerable populations, functional-based approach, access and functional needs (AFNs)'

In [16]:
data = data.assign(topics=data.abstract.apply(ask_chatgpt))

In [17]:
data

,authors,title,pub_date,abstract,cite_count,doi,pii,openaccess,topics
0,"Zhang, Fengxiu;Xiang, Tianyi",Attending to the unattended: Why and how do lo...,2024-12-31,Research and practice in climate risk reductio...,0,10.1016/j.envsci.2024.103892,S1462901124002260,false,Climate risk reduction and functional-based ap...
1,"Yoshida, Yuki;Sitas, Nadia;Mannetti, Lelani;O’...",Beyond Academia: A case for reviews of gray li...,2024-12-31,Gray literature is increasingly considered to ...,0,10.1016/j.envsci.2024.103882,S1462901124002168,true,gray literature review
2,"Pietrzyk-Kaszyńska, Agata;Olszańska, Agnieszka",Of heroes and villains – How coalitions shape ...,2024-12-31,Policy narrative analyses provide important in...,0,10.1016/j.envsci.2024.103899,S1462901124002338,false,"nature conservation, policy narratives, coalit..."
3,"Zurba, Melanie;Suchet-Pearson, Sandie;Bullock,...",Enhancing meaningful Indigenous leadership and...,2024-12-31,This is the first global empirical study that ...,0,10.1016/j.envsci.2024.103864,S1462901124001989,true,Indigenous participation in global environment...
4,"Lemke, Leonard Kwhang-Gil;Beier, Julia;Hanger-...",Exploring procedural justice in stakeholder id...,2024-12-31,"In the face of complex societal challenges, st...",0,10.1016/j.envsci.2024.103900,S146290112400234X,true,"stakeholder identification, procedural justice..."
...,...,...,...,...,...,...,...,...,...
3025,"Danila, Donald J",Estimating the abundance and egg production of...,2000-09-01,The winter flounder is a coastal flatfish with...,9,10.1016/S1462-9011(00)00059-9,S1462901100000599,false,Winter flounder population dynamics near nucle...
3026,"Lorda, Ernesto;Danila, Donald J.;Miller, John D.",Application of a population dynamics model to ...,2000-09-01,A major concern of the Connecticut Department ...,7,10.1016/S1462-9011(00)00061-7,S1462901100000617,false,impact of Cooling Water Intakes System (CWIS) ...
3027,"Jager, Henriëtte I;Lepla, Ken;Chandler, James;...",Population viability analysis of white sturgeo...,2000-09-01,Many utilities face conflicts between two goal...,13,10.1016/S1462-9011(00)00063-0,S1462901100000630,false,fish population modeling and management
3028,"Farber, Stephen",Welfare-based ecosystem management: an investi...,2000-09-01,This paper investigates the use of welfare-bas...,1,10.1016/S1462-9011(00)00066-6,S1462901100000666,false,"environmental management, economic criteria, t..."


In [19]:
data.to_excel('processed_by_LLMs.xlsx')